In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2024-01-13T10:11:54.237715-05:00

Python implementation: CPython
Python version       : 3.7.11
IPython version      : 7.10.1

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 4.15.0-161-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 40
Architecture: 64bit



In [3]:
%watermark --gpu

GPU Info: 
  GPU 0: Tesla V100-DGXS-32GB
  GPU 1: Tesla V100-DGXS-32GB
  GPU 2: Tesla V100-DGXS-32GB
  GPU 3: Tesla V100-DGXS-32GB



In [4]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import optuna
import gc
import logging

In [5]:
%watermark --iversions

dask   : 2022.1.0
xgboost: 1.6.2
pandas : 1.3.1
numpy  : 1.21.6
logging: 0.5.1.2
optuna : 2.10.1



In [6]:
!nvidia-smi -L

GPU 0: Tesla V100-DGXS-32GB (UUID: GPU-d752af09-1f62-bf3b-4f70-78b84e9e41f6)
GPU 1: Tesla V100-DGXS-32GB (UUID: GPU-054a4a35-f98a-3ebc-9100-0f697c246b43)
GPU 2: Tesla V100-DGXS-32GB (UUID: GPU-454525d4-bebd-7fb8-0ba3-3b85e2f99abd)
GPU 3: Tesla V100-DGXS-32GB (UUID: GPU-af0fb74e-f5eb-0833-17ff-494cc6bdcee1)


In [7]:
cluster = LocalCUDACluster(n_workers=4)
client = Client(cluster)

In [8]:
!nvidia-smi

Sat Jan 13 10:12:09 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0  On |                    0 |
| N/A   37C    P0    51W / 300W |   1339MiB / 32505MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   

In [9]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [10]:
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input/xgtrain_fold_{i}_drop_other.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input/xgval_fold_{i}_drop_other.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4


In [11]:
train = pd.read_csv('../input/train.csv.zip')

target = train['target'].values
target

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    kf = KFold(5, shuffle=True, random_state=137)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = predictions
        del dtrain, dval, output
        gc.collect()
        gc.collect()

    gini = gini_normalized(target, train_oof)
    
    return gini

In [13]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_4.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_porto_4.db", study_name="five_fold_optuna_xgb_4", direction='maximize')

In [14]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 50% CPU time recently (threshold: 10%)


CPU times: user 3min 34s, sys: 48.2 s, total: 4min 22s
Wall time: 45min 14s


In [15]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.278231,0.237959,0.307538,logloss,0.040181,0.014219,14,57,binary:logistic,0.479598,gpu_hist,COMPLETE
1,1,0.278840,0.122167,0.355686,logloss,0.024030,0.016764,25,143,binary:logistic,0.750736,gpu_hist,COMPLETE
2,2,0.270571,0.003283,0.907102,logloss,0.439070,0.030743,25,204,binary:logistic,0.561081,gpu_hist,COMPLETE


In [16]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_4.csv', index=False)
df.head(8)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 75% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 75% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 75% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 77% CPU time recently (thre

CPU times: user 3min 56s, sys: 49.8 s, total: 4min 45s
Wall time: 47min 42s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.278231,0.237959,0.307538,logloss,0.040181,0.014219,14,57,binary:logistic,0.479598,gpu_hist,COMPLETE
1,1,0.278840,0.122167,0.355686,logloss,0.024030,0.016764,25,143,binary:logistic,0.750736,gpu_hist,COMPLETE
2,2,0.270571,0.003283,0.907102,logloss,0.439070,0.030743,25,204,binary:logistic,0.561081,gpu_hist,COMPLETE
3,3,0.263175,0.282777,0.947388,logloss,1.141320,0.067313,6,275,binary:logistic,0.675927,gpu_hist,COMPLETE
4,4,0.238953,5.375599,0.529730,logloss,0.006652,0.001190,4,268,binary:logistic,0.665472,gpu_hist,COMPLETE
5,5,0.267708,0.136897,0.664017,logloss,0.690717,0.048935,21,270,binary:logistic,0.467052,gpu_hist,COMPLETE
6,6,0.284175,5.332065,0.572221,logloss,0.106138,0.010697,10,111,binary:logistic,0.926923,gpu_hist,COMPLETE
7,7,0.269233,0.001077,0.706192,logloss,0.002045,0.001020,25,18,binary:logistic,0.843782,gpu_hist,COMPLETE


In [17]:
study.best_trial.params

{'alpha': 5.332064506595186,
 'colsample_bytree': 0.5722213687634099,
 'eval_metric': 'logloss',
 'lambda': 0.10613754123896357,
 'learning_rate': 0.01069747259901222,
 'max_depth': 10,
 'min_child_weight': 111,
 'objective': 'binary:logistic',
 'subsample': 0.9269227767246805,
 'tree_method': 'gpu_hist'}

In [18]:
df.value.max()

0.2841752878869825

In [19]:
%%time
study.optimize(objective, n_trials=100)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_4.csv', index=False)
df.head(20)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 80% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 80% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 80% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 80% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 81% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 81% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 81% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 81% CPU time recently (thre

CPU times: user 32min 27s, sys: 5min 27s, total: 37min 54s
Wall time: 5h 11min 41s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.278231,0.237959,0.307538,logloss,0.040181,0.014219,14,57,binary:logistic,0.479598,gpu_hist,COMPLETE
1,1,0.278840,0.122167,0.355686,logloss,0.024030,0.016764,25,143,binary:logistic,0.750736,gpu_hist,COMPLETE
2,2,0.270571,0.003283,0.907102,logloss,0.439070,0.030743,25,204,binary:logistic,0.561081,gpu_hist,COMPLETE
3,3,0.263175,0.282777,0.947388,logloss,1.141320,0.067313,6,275,binary:logistic,0.675927,gpu_hist,COMPLETE
4,4,0.238953,5.375599,0.529730,logloss,0.006652,0.001190,4,268,binary:logistic,0.665472,gpu_hist,COMPLETE
5,5,0.267708,0.136897,0.664017,logloss,0.690717,0.048935,21,270,binary:logistic,0.467052,gpu_hist,COMPLETE
6,6,0.284175,5.332065,0.572221,logloss,0.106138,0.010697,10,111,binary:logistic,0.926923,gpu_hist,COMPLETE
7,7,0.269233,0.001077,0.706192,logloss,0.002045,0.001020,25,18,binary:logistic,0.843782,gpu_hist,COMPLETE
8,8,0.280260,6.491985,0.839929,logloss,0.020841,0.020519,3,169,binary:logistic,0.678270,gpu_hist,COMPLETE
9,9,0.274942,9.635088,0.925396,logloss,0.001813,0.047376,5,244,binary:logistic,0.495427,gpu_hist,COMPLETE


In [20]:
df.value.max()

0.2841752878869825

In [21]:
study.best_trial.params

{'alpha': 5.332064506595186,
 'colsample_bytree': 0.5722213687634099,
 'eval_metric': 'logloss',
 'lambda': 0.10613754123896357,
 'learning_rate': 0.01069747259901222,
 'max_depth': 10,
 'min_child_weight': 111,
 'objective': 'binary:logistic',
 'subsample': 0.9269227767246805,
 'tree_method': 'gpu_hist'}

In [22]:
%%time
study.optimize(objective, n_trials=100)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_4.csv', index=False)
df.head(20)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 91% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 92% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 92% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 92% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 92% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 92% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 92% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 92% CPU time recently (thre

CPU times: user 22min 29s, sys: 1min 45s, total: 24min 14s
Wall time: 1h 53min 37s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.278231,0.237959,0.307538,logloss,0.040181,0.014219,14,57,binary:logistic,0.479598,gpu_hist,COMPLETE
1,1,0.278840,0.122167,0.355686,logloss,0.024030,0.016764,25,143,binary:logistic,0.750736,gpu_hist,COMPLETE
2,2,0.270571,0.003283,0.907102,logloss,0.439070,0.030743,25,204,binary:logistic,0.561081,gpu_hist,COMPLETE
3,3,0.263175,0.282777,0.947388,logloss,1.141320,0.067313,6,275,binary:logistic,0.675927,gpu_hist,COMPLETE
4,4,0.238953,5.375599,0.529730,logloss,0.006652,0.001190,4,268,binary:logistic,0.665472,gpu_hist,COMPLETE
5,5,0.267708,0.136897,0.664017,logloss,0.690717,0.048935,21,270,binary:logistic,0.467052,gpu_hist,COMPLETE
6,6,0.284175,5.332065,0.572221,logloss,0.106138,0.010697,10,111,binary:logistic,0.926923,gpu_hist,COMPLETE
7,7,0.269233,0.001077,0.706192,logloss,0.002045,0.001020,25,18,binary:logistic,0.843782,gpu_hist,COMPLETE
8,8,0.280260,6.491985,0.839929,logloss,0.020841,0.020519,3,169,binary:logistic,0.678270,gpu_hist,COMPLETE
9,9,0.274942,9.635088,0.925396,logloss,0.001813,0.047376,5,244,binary:logistic,0.495427,gpu_hist,COMPLETE


In [23]:
study.best_trial.params

{'alpha': 5.332064506595186,
 'colsample_bytree': 0.5722213687634099,
 'eval_metric': 'logloss',
 'lambda': 0.10613754123896357,
 'learning_rate': 0.01069747259901222,
 'max_depth': 10,
 'min_child_weight': 111,
 'objective': 'binary:logistic',
 'subsample': 0.9269227767246805,
 'tree_method': 'gpu_hist'}

In [24]:
df.value.max()

0.2841752878869825